In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
import catboost
print(catboost.__version__)

0.26


In [2]:
## Reading the data 
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

sub = pd.read_csv('Submission.csv')

In [3]:
train.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [4]:
## Exploring the data 
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
train['bank_account']=lb.fit_transform(train['bank_account'])


In [5]:
# splitting our data into features and target.
y= train['bank_account']
X = train.drop(['bank_account'],axis=1)


In [6]:
# categorical features declaration
X.shape

(23524, 12)

In [7]:
X.columns

Index(['country', 'year', 'uniqueid', 'location_type', 'cellphone_access',
       'household_size', 'age_of_respondent', 'gender_of_respondent',
       'relationship_with_head', 'marital_status', 'education_level',
       'job_type'],
      dtype='object')

In [8]:
# lets have a list of categorical columns and feed it 
cat_cols =['country','uniqueid','location_type',
                'cellphone_access','gender_of_respondent',
               'relationship_with_head','marital_status',
               'education_level','job_type']

cat_features = X[cat_cols]
cat_features.head()
           
           

,country,uniqueid,location_type,cellphone_access,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,uniqueid_1,Rural,Yes,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,uniqueid_2,Rural,No,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,uniqueid_3,Urban,Yes,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,uniqueid_4,Rural,Yes,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,uniqueid_5,Urban,No,Male,Child,Single/Never Married,Primary education,Informally employed


In [9]:
# Looking on label balance on our target data
print('Labels:  {}  '.format(set(y)))

print('Zero count = {}  One count = {}'.format(len(y)-sum(y) ,sum(y)))



Labels:  {0, 1}  
Zero count = 20212  One count = 3312


In [10]:
even_rate = (len(y)-sum(y))/sum(y)
print('Even rate for this study is =  {} '.format(even_rate))

Even rate for this study is =  6.102657004830918 


In [11]:
## Training the first model 
from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=200,
                                            random_seed=21,
                                            )

model.fit(X,y,cat_features,
          verbose=10,
           plot=True,
             )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.17359
0:	learn: 0.5450090	total: 277ms	remaining: 55.1s
10:	learn: 0.2913613	total: 962ms	remaining: 16.5s
20:	learn: 0.2779081	total: 1.7s	remaining: 14.4s
30:	learn: 0.2729626	total: 2.44s	remaining: 13.3s
40:	learn: 0.2718974	total: 3.16s	remaining: 12.3s
50:	learn: 0.2699264	total: 3.88s	remaining: 11.3s
60:	learn: 0.2677399	total: 4.59s	remaining: 10.5s
70:	learn: 0.2662378	total: 5.29s	remaining: 9.61s
80:	learn: 0.2647836	total: 6.04s	remaining: 8.87s
90:	learn: 0.2633452	total: 6.82s	remaining: 8.17s
100:	learn: 0.2614547	total: 7.58s	remaining: 7.43s
110:	learn: 0.2602142	total: 8.35s	remaining: 6.69s
120:	learn: 0.2587047	total: 9.11s	remaining: 5.95s
130:	learn: 0.2573783	total: 9.8s	remaining: 5.16s
140:	learn: 0.2558625	total: 10.5s	remaining: 4.4s
150:	learn: 0.2542904	total: 11.3s	remaining: 3.67s
160:	learn: 0.2527844	total: 12.1s	remaining: 2.93s
170:	learn: 0.2510143	total: 12.8s	remaining: 2.17s
180:	learn: 0.2496679	total: 13.5s	remaining: 1.4

In [12]:
model.predict_proba(X)

array([[0.61639193, 0.38360807],
       [0.9236585 , 0.0763415 ],
       [0.24884742, 0.75115258],
       ...,
       [0.98086481, 0.01913519],
       [0.81149316, 0.18850684],
       [0.97751592, 0.02248408]])

In [13]:
# Let's use pool to split our data into train and validation data 
from catboost import Pool
pool = Pool(data=X,label=y,cat_features=cat_features)

In [14]:
from sklearn.model_selection import train_test_split 
data = train_test_split(X,y,test_size=0.2,random_state=21)
X_train,X_validation,y_train,y_validation = data 

# this is our training pool
train_pool = Pool(
data=X_train,
label =  y_train,
cat_features =  cat_features
)

# here is our validation pool
validation_pool = Pool(
data=X_validation,
label =  y_validation,
cat_features =  cat_features
)

In [15]:
## model_two using  CatBoostClassifier 
model_two = CatBoostClassifier( iterations=400,early_stopping_rounds=270,custom_loss=['AUC','Accuracy'])

model_two.fit(train_pool,eval_set=validation_pool,verbose=False,plot=True)

print('Model is fitted: {}'.format(model_two.is_fitted()))
print('Model params:\n {}'.format(model_two.get_params()))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Model is fitted: True
Model params:
 {'iterations': 400, 'custom_loss': ['AUC', 'Accuracy'], 'early_stopping_rounds': 270}


In [16]:
# best_iteration
modelThree = CatBoostClassifier(
    iterations=100,
    use_best_model = True
    
    

)


modelThree.fit(
    train_pool,
    eval_set = validation_pool,
    verbose=False,
    plot=True

)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [17]:
print('Best model tree count is {}'.format(modelThree.tree_count_))

Best model tree count is 79


In [18]:
## Attempting gridsearch
from catboost import cv
from sklearn.model_selection import GridSearchCV

model = CatBoostClassifier(loss_function='Logloss',
                                               cat_features = cat_features
                                              )

params  = {
    'iterations':[400,800,1000],
    # iterations signify the number of trees
    'depth': [5,10],
    'l2_leaf_reg': [1, 7, 9]      
} 



grid_search = GridSearchCV(
    model,
    param_grid=params,
    cv=3


)


results = grid_search.fit(X_train,y_train)

results.best_estimator_.get_params()


0:	learn: 0.6661638	total: 50.2ms	remaining: 20s
1:	learn: 0.6420591	total: 83.6ms	remaining: 16.6s
2:	learn: 0.6167328	total: 136ms	remaining: 18s
3:	learn: 0.5946044	total: 180ms	remaining: 17.8s
4:	learn: 0.5706456	total: 224ms	remaining: 17.7s
5:	learn: 0.5517454	total: 262ms	remaining: 17.2s
6:	learn: 0.5334539	total: 301ms	remaining: 16.9s
7:	learn: 0.5148695	total: 333ms	remaining: 16.3s
8:	learn: 0.5013649	total: 371ms	remaining: 16.1s
9:	learn: 0.4869236	total: 410ms	remaining: 16s
10:	learn: 0.4739705	total: 455ms	remaining: 16.1s
11:	learn: 0.4622438	total: 499ms	remaining: 16.1s
12:	learn: 0.4508483	total: 537ms	remaining: 16s
13:	learn: 0.4403844	total: 572ms	remaining: 15.8s
14:	learn: 0.4320204	total: 605ms	remaining: 15.5s
15:	learn: 0.4226021	total: 640ms	remaining: 15.4s
16:	learn: 0.4149769	total: 667ms	remaining: 15s
17:	learn: 0.4071374	total: 705ms	remaining: 15s
18:	learn: 0.3995802	total: 744ms	remaining: 14.9s
19:	learn: 0.3912645	total: 777ms	remaining: 14.8s


164:	learn: 0.2739736	total: 6.64s	remaining: 9.46s
165:	learn: 0.2739407	total: 6.68s	remaining: 9.42s
166:	learn: 0.2738473	total: 6.72s	remaining: 9.38s
167:	learn: 0.2737976	total: 6.77s	remaining: 9.35s
168:	learn: 0.2737274	total: 6.81s	remaining: 9.31s
169:	learn: 0.2735839	total: 6.85s	remaining: 9.27s
170:	learn: 0.2735048	total: 6.9s	remaining: 9.24s
171:	learn: 0.2734440	total: 6.94s	remaining: 9.2s
172:	learn: 0.2734258	total: 6.97s	remaining: 9.14s
173:	learn: 0.2733474	total: 7.01s	remaining: 9.11s
174:	learn: 0.2730962	total: 7.05s	remaining: 9.07s
175:	learn: 0.2730275	total: 7.09s	remaining: 9.03s
176:	learn: 0.2729531	total: 7.14s	remaining: 8.99s
177:	learn: 0.2728585	total: 7.18s	remaining: 8.96s
178:	learn: 0.2728034	total: 7.22s	remaining: 8.92s
179:	learn: 0.2727170	total: 7.27s	remaining: 8.88s
180:	learn: 0.2726863	total: 7.3s	remaining: 8.84s
181:	learn: 0.2725147	total: 7.34s	remaining: 8.79s
182:	learn: 0.2724548	total: 7.39s	remaining: 8.76s
183:	learn: 0.2

327:	learn: 0.2647892	total: 13.5s	remaining: 2.96s
328:	learn: 0.2647635	total: 13.5s	remaining: 2.92s
329:	learn: 0.2647448	total: 13.6s	remaining: 2.88s
330:	learn: 0.2647281	total: 13.6s	remaining: 2.84s
331:	learn: 0.2646846	total: 13.7s	remaining: 2.8s
332:	learn: 0.2645926	total: 13.7s	remaining: 2.76s
333:	learn: 0.2645615	total: 13.7s	remaining: 2.72s
334:	learn: 0.2644720	total: 13.8s	remaining: 2.68s
335:	learn: 0.2644299	total: 13.8s	remaining: 2.63s
336:	learn: 0.2643985	total: 13.9s	remaining: 2.59s
337:	learn: 0.2643733	total: 13.9s	remaining: 2.55s
338:	learn: 0.2643490	total: 14s	remaining: 2.51s
339:	learn: 0.2643200	total: 14s	remaining: 2.47s
340:	learn: 0.2641926	total: 14.1s	remaining: 2.43s
341:	learn: 0.2641509	total: 14.1s	remaining: 2.39s
342:	learn: 0.2640781	total: 14.1s	remaining: 2.35s
343:	learn: 0.2640557	total: 14.2s	remaining: 2.31s
344:	learn: 0.2640482	total: 14.2s	remaining: 2.27s
345:	learn: 0.2640009	total: 14.3s	remaining: 2.23s
346:	learn: 0.263

91:	learn: 0.2890220	total: 3.65s	remaining: 12.2s
92:	learn: 0.2888696	total: 3.69s	remaining: 12.2s
93:	learn: 0.2886980	total: 3.73s	remaining: 12.1s
94:	learn: 0.2885019	total: 3.77s	remaining: 12.1s
95:	learn: 0.2880774	total: 3.81s	remaining: 12.1s
96:	learn: 0.2877012	total: 3.86s	remaining: 12.1s
97:	learn: 0.2873792	total: 3.9s	remaining: 12s
98:	learn: 0.2870208	total: 3.94s	remaining: 12s
99:	learn: 0.2866672	total: 3.98s	remaining: 11.9s
100:	learn: 0.2864953	total: 4.01s	remaining: 11.9s
101:	learn: 0.2863981	total: 4.05s	remaining: 11.8s
102:	learn: 0.2862307	total: 4.09s	remaining: 11.8s
103:	learn: 0.2860792	total: 4.13s	remaining: 11.7s
104:	learn: 0.2858977	total: 4.17s	remaining: 11.7s
105:	learn: 0.2856271	total: 4.21s	remaining: 11.7s
106:	learn: 0.2852662	total: 4.25s	remaining: 11.7s
107:	learn: 0.2852272	total: 4.29s	remaining: 11.6s
108:	learn: 0.2848782	total: 4.34s	remaining: 11.6s
109:	learn: 0.2847551	total: 4.38s	remaining: 11.5s
110:	learn: 0.2846294	tota

250:	learn: 0.2740386	total: 10.1s	remaining: 5.98s
251:	learn: 0.2740109	total: 10.1s	remaining: 5.95s
252:	learn: 0.2739866	total: 10.2s	remaining: 5.91s
253:	learn: 0.2739704	total: 10.2s	remaining: 5.87s
254:	learn: 0.2739332	total: 10.3s	remaining: 5.83s
255:	learn: 0.2739193	total: 10.3s	remaining: 5.79s
256:	learn: 0.2739069	total: 10.3s	remaining: 5.75s
257:	learn: 0.2738451	total: 10.4s	remaining: 5.71s
258:	learn: 0.2738170	total: 10.4s	remaining: 5.67s
259:	learn: 0.2737625	total: 10.4s	remaining: 5.63s
260:	learn: 0.2737539	total: 10.5s	remaining: 5.58s
261:	learn: 0.2736399	total: 10.5s	remaining: 5.54s
262:	learn: 0.2735286	total: 10.6s	remaining: 5.51s
263:	learn: 0.2735223	total: 10.6s	remaining: 5.46s
264:	learn: 0.2734770	total: 10.7s	remaining: 5.43s
265:	learn: 0.2734519	total: 10.7s	remaining: 5.39s
266:	learn: 0.2734415	total: 10.7s	remaining: 5.35s
267:	learn: 0.2734136	total: 10.8s	remaining: 5.3s
268:	learn: 0.2733470	total: 10.8s	remaining: 5.27s
269:	learn: 0

...

801:	learn: 0.1819867	total: 1m 13s	remaining: 8.98s
802:	learn: 0.1819118	total: 1m 13s	remaining: 8.89s
803:	learn: 0.1817843	total: 1m 13s	remaining: 8.8s
804:	learn: 0.1817050	total: 1m 13s	remaining: 8.71s
805:	learn: 0.1814922	total: 1m 13s	remaining: 8.62s
806:	learn: 0.1814600	total: 1m 13s	remaining: 8.53s
807:	learn: 0.1813551	total: 1m 14s	remaining: 8.44s
808:	learn: 0.1810007	total: 1m 14s	remaining: 8.35s
809:	learn: 0.1809633	total: 1m 14s	remaining: 8.25s
810:	learn: 0.1809617	total: 1m 14s	remaining: 8.16s
811:	learn: 0.1808974	total: 1m 14s	remaining: 8.07s
812:	learn: 0.1808015	total: 1m 14s	remaining: 7.97s
813:	learn: 0.1806588	total: 1m 14s	remaining: 7.88s
814:	learn: 0.1806172	total: 1m 14s	remaining: 7.79s
815:	learn: 0.1806151	total: 1m 14s	remaining: 7.7s
816:	learn: 0.1806137	total: 1m 14s	remaining: 7.6s
817:	learn: 0.1805594	total: 1m 14s	remaining: 7.51s
818:	learn: 0.1802842	total: 1m 15s	remaining: 7.42s
819:	learn: 0.1800985	total: 1m 15s	remaining: 7.

62:	learn: 0.2984878	total: 1.79s	remaining: 9.57s
63:	learn: 0.2979870	total: 1.82s	remaining: 9.54s
64:	learn: 0.2974873	total: 1.84s	remaining: 9.49s
65:	learn: 0.2965230	total: 1.87s	remaining: 9.47s
66:	learn: 0.2960608	total: 1.9s	remaining: 9.44s
67:	learn: 0.2955586	total: 1.92s	remaining: 9.39s
68:	learn: 0.2949483	total: 1.96s	remaining: 9.38s
69:	learn: 0.2945391	total: 1.98s	remaining: 9.35s
70:	learn: 0.2942252	total: 2.01s	remaining: 9.33s
71:	learn: 0.2935451	total: 2.05s	remaining: 9.32s
72:	learn: 0.2931774	total: 2.08s	remaining: 9.34s
73:	learn: 0.2928666	total: 2.11s	remaining: 9.31s
74:	learn: 0.2925272	total: 2.15s	remaining: 9.3s
75:	learn: 0.2921524	total: 2.19s	remaining: 9.34s
76:	learn: 0.2918157	total: 2.22s	remaining: 9.32s
77:	learn: 0.2912542	total: 2.25s	remaining: 9.28s
78:	learn: 0.2907389	total: 2.28s	remaining: 9.26s
79:	learn: 0.2904921	total: 2.31s	remaining: 9.25s
80:	learn: 0.2900680	total: 2.35s	remaining: 9.25s
81:	learn: 0.2898846	total: 2.39s

225:	learn: 0.2753420	total: 6.78s	remaining: 5.22s
226:	learn: 0.2753258	total: 6.81s	remaining: 5.19s
227:	learn: 0.2752142	total: 6.84s	remaining: 5.16s
228:	learn: 0.2751913	total: 6.87s	remaining: 5.13s
229:	learn: 0.2751314	total: 6.89s	remaining: 5.09s
230:	learn: 0.2750713	total: 6.92s	remaining: 5.06s
231:	learn: 0.2750466	total: 6.96s	remaining: 5.04s
232:	learn: 0.2750254	total: 6.99s	remaining: 5.01s
233:	learn: 0.2749943	total: 7.02s	remaining: 4.98s
234:	learn: 0.2749453	total: 7.05s	remaining: 4.95s
235:	learn: 0.2749084	total: 7.08s	remaining: 4.92s
236:	learn: 0.2748956	total: 7.1s	remaining: 4.89s
237:	learn: 0.2748352	total: 7.13s	remaining: 4.86s
238:	learn: 0.2748298	total: 7.15s	remaining: 4.82s
239:	learn: 0.2748128	total: 7.17s	remaining: 4.78s
240:	learn: 0.2747900	total: 7.21s	remaining: 4.76s
241:	learn: 0.2747476	total: 7.24s	remaining: 4.72s
242:	learn: 0.2747129	total: 7.26s	remaining: 4.69s
243:	learn: 0.2747045	total: 7.29s	remaining: 4.66s
244:	learn: 0

385:	learn: 0.2701649	total: 11.8s	remaining: 428ms
386:	learn: 0.2701463	total: 11.8s	remaining: 397ms
387:	learn: 0.2701175	total: 11.9s	remaining: 367ms
388:	learn: 0.2700080	total: 11.9s	remaining: 337ms
389:	learn: 0.2699548	total: 11.9s	remaining: 306ms
390:	learn: 0.2699415	total: 12s	remaining: 276ms
391:	learn: 0.2699214	total: 12s	remaining: 246ms
392:	learn: 0.2698929	total: 12.1s	remaining: 215ms
393:	learn: 0.2698736	total: 12.1s	remaining: 184ms
394:	learn: 0.2698141	total: 12.1s	remaining: 154ms
395:	learn: 0.2697868	total: 12.2s	remaining: 123ms
396:	learn: 0.2697778	total: 12.2s	remaining: 92.3ms
397:	learn: 0.2697592	total: 12.2s	remaining: 61.5ms
398:	learn: 0.2697007	total: 12.3s	remaining: 30.8ms
399:	learn: 0.2696763	total: 12.3s	remaining: 0us


{'iterations': 400,
 'depth': 5,
 'l2_leaf_reg': 1,
 'loss_function': 'Logloss',
 'cat_features':       country       uniqueid location_type cellphone_access  \
 0       Kenya     uniqueid_1         Rural              Yes   
 1       Kenya     uniqueid_2         Rural               No   
 2       Kenya     uniqueid_3         Urban              Yes   
 3       Kenya     uniqueid_4         Rural              Yes   
 4       Kenya     uniqueid_5         Urban               No   
 ...       ...            ...           ...              ...   
 23519  Uganda  uniqueid_2113         Rural              Yes   
 23520  Uganda  uniqueid_2114         Rural              Yes   
 23521  Uganda  uniqueid_2115         Rural              Yes   
 23522  Uganda  uniqueid_2116         Urban              Yes   
 23523  Uganda  uniqueid_2117         Rural              Yes   
 
       gender_of_respondent relationship_with_head           marital_status  \
 0                   Female                 Spouse  Ma

In [24]:
model = CatBoostClassifier(loss_function='Logloss',
                                               cat_features = cat_features,
                                               iterations = 400,
                                               l2_leaf_reg = 1,
                                               depth = 5
                                               
                                              )

model.fit( train_pool,
    eval_set = validation_pool,
    verbose=False,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [25]:
test.head()

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_6056,Urban,Yes,3,30,Male,Head of Household,Married/Living together,Secondary education,Formally employed Government
1,Kenya,2018,uniqueid_6060,Urban,Yes,7,51,Male,Head of Household,Married/Living together,Vocational/Specialised training,Formally employed Private
2,Kenya,2018,uniqueid_6065,Rural,No,3,77,Female,Parent,Married/Living together,No formal education,Remittance Dependent
3,Kenya,2018,uniqueid_6072,Rural,No,6,39,Female,Head of Household,Married/Living together,Primary education,Remittance Dependent
4,Kenya,2018,uniqueid_6073,Urban,No,3,16,Male,Child,Single/Never Married,Secondary education,Remittance Dependent


In [26]:
print(X.shape)
print(test.shape)

(23524, 12)
(10086, 12)


In [27]:
testId = test['uniqueid']
testCountry = test['country']
test_pool = Pool(data=test, cat_features=cat_features)
contest_predictions = model.predict_proba(test_pool)
print('Predictoins:')
print(contest_predictions)

contpreds = model.predict(test_pool)
print(contpreds)

Predictoins:
[[0.14231349 0.85768651]
 [0.14499756 0.85500244]
 [0.97979608 0.02020392]
 ...
 [0.68876064 0.31123936]
 [0.96744462 0.03255538]
 [0.9154143  0.0845857 ]]
[1 1 0 ... 0 0 0]


In [28]:
#convert the predictions to pandas dataframe
submission = pd.DataFrame(data={'uniqueid':testId+' x '+ testCountry,'bank_account':contpreds})

submission.to_csv('Submission.csv',index=False)